### SimpleTest_SPO_GraphFrames
    0_Get the file
    1_Building the Graph
        1.0_Create Vertices
        1.1_Create Edges
        1.2_Test Graph

In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [19]:
from pyspark.sql.functions import *
from graphframes import *

### 0_get the file

In [163]:
FilePath_SPO = "/Users/jeanxu/PycharmProjects/QUEST_TerminalTest/Code/files/SPO_paragraph_ques-q1.txt"

In [164]:
Trip_SPO = spark.read.csv(FilePath_SPOFilePath, header='False', inferSchema='true', sep='|')

In [165]:
Trip_SPO.show(10)

+------+-----------------+--------+--------------------+---+--------------------+----+--------------------+
|   _c0|              _c1|     _c2|                 _c3|_c4|                 _c5| _c6|                 _c7|
+------+-----------------+--------+--------------------+---+--------------------+----+--------------------+
|doc-0 | ./docs/doc1.txt | sent-0 |    navigation Jump |2.0|             search | 2.0|             article|
|doc-0 | ./docs/doc1.txt | sent-0 | free encyclopedi...|4.0|             search | 2.0|             article|
|doc-0 | ./docs/doc1.txt | sent-0 |          Wikipedia |7.0|             search | 2.0|             article|
|doc-0 | ./docs/doc1.txt | sent-0 |            article |1.0|           is about | 2.0|              person|
|doc-0 | ./docs/doc1.txt | sent-1 |         other uses |2.0|                see | 1.0| Steve Jobs disam...|
|doc-0 | ./docs/doc1.txt | sent-1 |         other uses |2.0|                see | 3.0|          co-founder|
|doc-0 | ./docs/doc1.txt | s

In [140]:
# def single_space(col):
#     return F.trim(F.regexp_replace(col, " +", " "))

In [166]:
Trip_SPO = Trip_SPO.withColumn("_c0", trim(col("_c0"))).withColumn("_c1", trim(col("_c1"))).withColumn("_c2", trim(col("_c2")))


In [167]:
Trip_SPO.show(2)

+-----+---------------+------+--------------------+---+--------+---+--------+
|  _c0|            _c1|   _c2|                 _c3|_c4|     _c5|_c6|     _c7|
+-----+---------------+------+--------------------+---+--------+---+--------+
|doc-0|./docs/doc1.txt|sent-0|    navigation Jump |2.0| search |2.0| article|
|doc-0|./docs/doc1.txt|sent-0| free encyclopedi...|4.0| search |2.0| article|
+-----+---------------+------+--------------------+---+--------+---+--------+
only showing top 2 rows



In [168]:
Trip_SPO = Trip_SPO.withColumn("_c3", trim(col("_c3"))).withColumn("_c4", trim(col("_c4"))).withColumn("_c5", trim(col("_c5")))

In [169]:
Trip_SPO.show(2)

+-----+---------------+------+--------------------+---+------+---+--------+
|  _c0|            _c1|   _c2|                 _c3|_c4|   _c5|_c6|     _c7|
+-----+---------------+------+--------------------+---+------+---+--------+
|doc-0|./docs/doc1.txt|sent-0|     navigation Jump|2.0|search|2.0| article|
|doc-0|./docs/doc1.txt|sent-0|free encyclopedia...|4.0|search|2.0| article|
+-----+---------------+------+--------------------+---+------+---+--------+
only showing top 2 rows



In [170]:
Trip_SPO = Trip_SPO.withColumn("_c6", trim(col("_c6"))).withColumn("_c7", trim(col("_c7")))

In [171]:
Trip_SPO.show(2)

+-----+---------------+------+--------------------+---+------+---+-------+
|  _c0|            _c1|   _c2|                 _c3|_c4|   _c5|_c6|    _c7|
+-----+---------------+------+--------------------+---+------+---+-------+
|doc-0|./docs/doc1.txt|sent-0|     navigation Jump|2.0|search|2.0|article|
|doc-0|./docs/doc1.txt|sent-0|free encyclopedia...|4.0|search|2.0|article|
+-----+---------------+------+--------------------+---+------+---+-------+
only showing top 2 rows



#### 0.0_rename the column name

In [172]:
Trip_SPO = Trip_SPO.withColumnRenamed("_c0", "document")
# select(column("_c0").alias("document"))

In [174]:
Trip_SPO.show(2)

+--------+---------------+------+--------------------+---+------+---+-------+
|document|            _c1|   _c2|                 _c3|_c4|   _c5|_c6|    _c7|
+--------+---------------+------+--------------------+---+------+---+-------+
|   doc-0|./docs/doc1.txt|sent-0|     navigation Jump|2.0|search|2.0|article|
|   doc-0|./docs/doc1.txt|sent-0|free encyclopedia...|4.0|search|2.0|article|
+--------+---------------+------+--------------------+---+------+---+-------+
only showing top 2 rows



In [175]:
Trip_SPO = Trip_SPO.withColumnRenamed("_c1", "path").withColumnRenamed("_c2", "sentence")

In [176]:
Trip_SPO = Trip_SPO.withColumnRenamed("_c3", "s1").withColumnRenamed("_c4", "d1").withColumnRenamed("_c5", "p").withColumnRenamed("_c6", "d2").withColumnRenamed("_c7", "s2")

In [177]:
Trip_SPO.show(2)


+--------+---------------+--------+--------------------+---+------+---+-------+
|document|           path|sentence|                  s1| d1|     p| d2|     s2|
+--------+---------------+--------+--------------------+---+------+---+-------+
|   doc-0|./docs/doc1.txt|  sent-0|     navigation Jump|2.0|search|2.0|article|
|   doc-0|./docs/doc1.txt|  sent-0|free encyclopedia...|4.0|search|2.0|article|
+--------+---------------+--------+--------------------+---+------+---+-------+
only showing top 2 rows



In [178]:
Trip_SPO.createOrReplaceTempView("Trip_SPO")

In [179]:
Trip_SPO.cache()

DataFrame[document: string, path: string, sentence: string, s1: string, d1: string, p: string, d2: string, s2: string]

### 1_building the Graph

#### 1.0_Create Vertices

In [180]:
# tripID_L = spark.sql("select distinct id from (select distinct s1 as id from Trip_SPO union all select distinct p as id from Trip_SPO) a")
tripID = Trip_SPO.select("s1").distinct().union(Trip_SPO.select("p").distinct()).union(Trip_SPO.select("s2").distinct()).distinct()

In [181]:
tripID = tripID.withColumnRenamed("s1","id")

In [182]:
tripID.show(10)

+--------------------+
|                  id|
+--------------------+
|Joanne Carole Sch...|
|Zen master Kōbun ...|
|          short time|
|graphical user in...|
|Amateur radio peo...|
|     confidentiality|
|           machinist|
|       Edwin H. Land|
|Health & Medicine...|
|business angel Mi...|
+--------------------+
only showing top 10 rows



In [183]:
tripID.count()

6591

In [184]:
tripID.createOrReplaceTempView("tripID")

In [187]:
tripID.where("id='Steve Jobs'").show()

+----------+
|        id|
+----------+
|Steve Jobs|
+----------+



#### [NOTE] a problem will be gerenated if do not exit the previous hive 
    Problem: "Unable to instantiate org.apache.hadoop.hive.ql.metadata.SessionHiveMetaStoreClient"

#### 1.1_Create Edges

In [188]:
tripEdges_L = Trip_SPO.select("s1", "p").distinct()

In [189]:
tripEdges_L.show(10)

+--------------------+--------------------+
|                  s1|                   p|
+--------------------+--------------------+
|                once|                told|
|            Polaroid|          said about|
|                 one|          said about|
|freshman English ...|          working on|
|              period|            slept on|
|                Jobs|          engaged in|
|     Larry Waterland|chemical engineer...|
|             Wozniak|             oversaw|
|               Clara|           motivated|
|             designs|              worked|
+--------------------+--------------------+
only showing top 10 rows



In [190]:
tripEdges_L = tripEdges_L.withColumnRenamed("s1","src").withColumnRenamed("p","dst")
tripEdges_L.show(2)

+--------+----------+
|     src|       dst|
+--------+----------+
|    once|      told|
|Polaroid|said about|
+--------+----------+
only showing top 2 rows



In [191]:
tripEdges_R = Trip_SPO.select("p", "s2").distinct()

In [192]:
tripEdges_R.show(10)

+----------------+--------------------+
|               p|                  s2|
+----------------+--------------------+
|             was|          co-founder|
|       member of|The Walt Disney C...|
|traveled through|               India|
|             was|                cars|
|       middle of|       seventh grade|
|        allowing|                Jobs|
|         decided|       John McCollum|
| smartest guy in|             Wozniak|
|            gave|               $ 700|
|         idea of|   daily convenience|
+----------------+--------------------+
only showing top 10 rows



In [193]:
tripEdges_R = tripEdges_R.withColumnRenamed("p","src").withColumnRenamed("s2","dst")
tripEdges_R.show(2)

+---------+--------------------+
|      src|                 dst|
+---------+--------------------+
|      was|          co-founder|
|member of|The Walt Disney C...|
+---------+--------------------+
only showing top 2 rows



In [194]:
tripEdges_L.createOrReplaceTempView("tripEdges_L")
tripEdges_R.createOrReplaceTempView("tripEdges_R")

In [195]:
tripID.count()

6591

In [202]:
# tripEdges = spark.sql("select f.src, f.dst from tripEdges_L f join tripID t on t.id = f.src")
tripEdges = tripEdges_L.select("src","dst").distinct().union(tripEdges_R.select("src","dst").distinct()).distinct()

In [203]:
tripEdges.show(10)

+--------------------+--------------------+
|                 src|                 dst|
+--------------------+--------------------+
|                once|                told|
|            Polaroid|          said about|
|                 one|          said about|
|freshman English ...|          working on|
|              period|            slept on|
|                Jobs|          engaged in|
|     Larry Waterland|chemical engineer...|
|             Wozniak|             oversaw|
|               Clara|           motivated|
|             designs|              worked|
+--------------------+--------------------+
only showing top 10 rows



In [204]:
tripEdges.count()

15079

In [215]:
# Cache Vertices and Edges
tripID.cache()
tripEdges.cache()

DataFrame[src: string, dst: string]

In [206]:
tripGraph = GraphFrame(tripID, tripEdges)

#### 1.2 test graph

In [207]:
tripGraph.edges.show(10)

+--------------------+--------------------+
|                 src|                 dst|
+--------------------+--------------------+
|                once|                told|
|            Polaroid|          said about|
|                 one|          said about|
|freshman English ...|          working on|
|              period|            slept on|
|                Jobs|          engaged in|
|     Larry Waterland|chemical engineer...|
|             Wozniak|             oversaw|
|               Clara|           motivated|
|             designs|              worked|
+--------------------+--------------------+
only showing top 10 rows



In [208]:
tripGraph.vertices.show(10)

+--------------------+
|                  id|
+--------------------+
|Joanne Carole Sch...|
|Zen master Kōbun ...|
|          short time|
|graphical user in...|
|Amateur radio peo...|
|     confidentiality|
|           machinist|
|       Edwin H. Land|
|Health & Medicine...|
|business angel Mi...|
+--------------------+
only showing top 10 rows



In [216]:
filteredPaths = tripGraph.bfs(
  fromExpr = "id = 'Steve Jobs'",
  toExpr = "id = 'born'",
  maxPathLength = 1000)
# FileterTest1 = tripGraph.edges.filter("src = 'Steve Jobs'").show(10)

In [217]:
filteredPaths.show()

+------------+------------------+------+
|        from|                e0|    to|
+------------+------------------+------+
|[Steve Jobs]|[Steve Jobs, born]|[born]|
+------------+------------------+------+



In [220]:
Jobs_Test = tripGraph.edges.filter("src = 'Steve Jobs'").show()

+----------+--------------------+
|       src|                 dst|
+----------+--------------------+
|Steve Jobs|Respiratory Arres...|
|Steve Jobs|         acknowledge|
|Steve Jobs|           published|
|Steve Jobs|        Evolution of|
|Steve Jobs|Lost Steve Jobs S...|
|Steve Jobs|             oversaw|
|Steve Jobs|           suspected|
|Steve Jobs|              buried|
|Steve Jobs|          noted that|
|Steve Jobs|        chronicle of|
|Steve Jobs| landmark deals with|
|Steve Jobs|                were|
|Steve Jobs|      Apple Board of|
|Steve Jobs|          married in|
|Steve Jobs|                told|
|Steve Jobs|      designed after|
|Steve Jobs|           vision of|
|Steve Jobs|  Apple community at|
|Steve Jobs|         focusing on|
|Steve Jobs|              remain|
+----------+--------------------+
only showing top 20 rows



In [221]:
Jobs_Test = tripGraph.edges.filter("src = 'born'").show()

+----+--------------------+
| src|                 dst|
+----+--------------------+
|born|          Steve Jobs|
|born|               world|
|born|      Joanne Simpson|
|born|                U.S.|
|born|  February 24 , 1955|
|born|two University of...|
|born|                1978|
|born|Abdulfattah John ...|
|born|      September 1991|
|born|                1955|
|born|     Joanne Schieble|
|born|        birth mother|
+----+--------------------+

